In [3]:
from cassandra.cluster import Cluster
from ssl import SSLContext, PROTOCOL_TLSv1_2, CERT_REQUIRED
from cassandra_sigv4.auth import SigV4AuthProvider
import boto3

# ssl setup
ssl_context = SSLContext(PROTOCOL_TLSv1_2)
ssl_context.load_verify_locations('/home/ubuntu/sf-class2-root.crt')  # change your file path for locating the certificate
ssl_context.verify_mode = CERT_REQUIRED

# boto3 session setup
boto_session = boto3.Session(region_name="us-east-2")  # this AWS credentials is specific to `us-east-2` region

/tmp/ipykernel_2103/3323863445.py:7: DeprecationWarning: ssl.PROTOCOL_TLSv1_2 is deprecated
  ssl_context = SSLContext(PROTOCOL_TLSv1_2)


In [4]:
# authorization setup with SigV4
auth_provider = SigV4AuthProvider(boto_session)

In [5]:
#cluster setup 
cluster = Cluster(['cassandra.us-east-2.amazonaws.com'], 
                  ssl_context=ssl_context, 
                  auth_provider=auth_provider, 
                  port=9142)  # TLS only communicates on port 9142

In [6]:
# establishing connection to Keyspace
session = cluster.connect()
# Insert any CQL queries between .connect() and .shutdown()

In [9]:
# For example, show all keyspaces created
r = session.execute('''
    SELECT * FROM system_schema.keyspaces;
    ''')
print(r.current_rows)

[Row(keyspace_name='system_schema', durable_writes=True, replication=OrderedMapSerializedKey([('class', 'org.apache.cassandra.locator.SimpleStrategy'), ('replication_factor', '3')])), Row(keyspace_name='system_schema_mcs', durable_writes=True, replication=OrderedMapSerializedKey([('class', 'org.apache.cassandra.locator.SimpleStrategy'), ('replication_factor', '3')])), Row(keyspace_name='system', durable_writes=True, replication=OrderedMapSerializedKey([('class', 'org.apache.cassandra.locator.SimpleStrategy'), ('replication_factor', '3')])), Row(keyspace_name='system_multiregion_info', durable_writes=True, replication=OrderedMapSerializedKey([('class', 'org.apache.cassandra.locator.SimpleStrategy'), ('replication_factor', '3')])), Row(keyspace_name='aet7207_hw2', durable_writes=True, replication=OrderedMapSerializedKey([('class', 'org.apache.cassandra.locator.SimpleStrategy'), ('replication_factor', '3')])), Row(keyspace_name='atb2199_ks', durable_writes=True, replication=OrderedMapSeri

In [8]:
# For example, create a keyspace for HW2
r = session.execute('''
    CREATE KEYSPACE IF NOT EXISTS csn4634_hw2 
    WITH replication = {'class': 'SingleRegionStrategy'};
    ''')
print(r.current_rows)

[]


In [10]:
from cassandra.cluster import ExecutionProfile, EXEC_PROFILE_DEFAULT
from cassandra import ConsistencyLevel


# Define execution profile with LOCAL_QUORUM
execution_profile = ExecutionProfile(
    consistency_level=ConsistencyLevel.LOCAL_QUORUM
)

# Cluster setup with correct profile
cluster = Cluster(
    ['cassandra.us-east-2.amazonaws.com'],
    ssl_context=ssl_context,
    auth_provider=auth_provider,
    port=9142,
    execution_profiles={EXEC_PROFILE_DEFAULT: execution_profile}
)

# establishing connection to Keyspace
session = cluster.connect()
session.set_keyspace('csn4634_hw2')  

In [12]:
# creating table
session.execute("""
CREATE TABLE IF NOT EXISTS q1 (
    ethnicity TEXT PRIMARY KEY,
    drug TEXT,
    count INT,
);
""")

In [20]:
# time to fill table
import pandas as pd
prs = pd.read_csv("Data/PRESCRIPTIONS.csv")
ads = pd.read_csv("Data/ADMISSIONS.csv")

combo = pd.merge(prs, ads[['subject_id', 'hadm_id', 'ethnicity']], on=['subject_id', 'hadm_id'], how = 'inner')
counts = combo.groupby(['ethnicity', 'drug']).size().reset_index(name = 'count')
top_counts = counts.sort_values(['ethnicity','count'], ascending = [True,False]).groupby('ethnicity').first().reset_index()

#insert row by row
for _, row in top_counts.iterrows():
    session.execute(
        """
        INSERT INTO q1 (ethnicity, drug, count)
        VALUES (%s, %s, %s)
        """,
        (row['ethnicity'], row['drug'], int(row['count']))
    )


In [26]:
# now query
ethnicities =session.execute(""" SELECT * FROM q1 """)
for ethnicity in ethnicities:
    print("The most common drug for those of ethnicity {" + ethnicity.ethnicity + "} is {" + ethnicity.drug + "}")

The most common drug for those of ethnicity {OTHER} is {NS}
The most common drug for those of ethnicity {BLACK/AFRICAN AMERICAN} is {Insulin}
The most common drug for those of ethnicity {WHITE} is {Potassium Chloride}
The most common drug for those of ethnicity {ASIAN} is {D5W}
The most common drug for those of ethnicity {HISPANIC/LATINO - PUERTO RICAN} is {0.9% Sodium Chloride}
The most common drug for those of ethnicity {UNKNOWN/NOT SPECIFIED} is {D5W}
The most common drug for those of ethnicity {UNABLE TO OBTAIN} is {0.9% Sodium Chloride}
The most common drug for those of ethnicity {AMERICAN INDIAN/ALASKA NATIVE FEDERALLY RECOGNIZED TRIBE} is {5% Dextrose}
The most common drug for those of ethnicity {HISPANIC OR LATINO} is {5% Dextrose}


In [ ]:
# create table
session.execute("""
CREATE TABLE IF NOT EXISTS q1 (
    ethnicity TEXT PRIMARY KEY,
    drug TEXT,
    count INT,
);
""")